In [20]:
import pygame
pygame.init()
font = pygame.font.SysFont('tahoma', 20)

BLACK = (0, 0, 0)
PURPLE = (255, 0, 255)
WHITE = (255, 255, 255)
WIDTH, HEIGHT = 800, 700
GRAVITY = 0.1

screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("DKY")

class Player:
    def __init__(self, pos, radius, speed, color):
        self.pos = pos
        self.radius = radius
        self.speed = speed
        self.color = color 

    def display(self):
        pygame.draw.circle(screen, self.color, self.pos, self.radius)

    def update(self):
        self.pos[0] += self.speed[0]
        self.pos[1] += self.speed[1]
        self.speed[1] += GRAVITY
        
class Wall:
    def __init__(self, begin, end, color):
        self.begin = begin
        self.end = end
        self.color = color

    def display(self):
        pygame.draw.line(screen, self.color, self.begin, self.end) 

 

def main():
    running = True

    player = Player([100, 100], 10, [0, 0], PURPLE)
    wall1 = Wall((50, 200), (400, 200), WHITE)
    wall2 = Wall((400, 200), (400, 600) , WHITE)
    
    walls = [wall1, wall2]
    
    while running:
        screen.fill(BLACK)
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        for wall in walls:
            wall.display()
        
        player.update()
        player.display()

        pygame.display.update()


if __name__ == "__main__":
    main()
    pygame.quit()